# RP24 analysis and 2D graphics


The objective of this example is to present problem 24 of the BBRC. We also present graphic elements for the visualization of the limit state surface in 2 dimensions.

In [ ]:
import openturns as ot
import numpy as np
import otbenchmark as otb

In [ ]:
problem = otb.ReliabilityProblem24()

In [ ]:
print(problem)

In [ ]:
event = problem.getEvent()
g = event.getFunction()

In [ ]:
problem.getProbability()

In [ ]:
# Create the Monte-Carlo algorithm
algoProb = ot.ProbabilitySimulationAlgorithm(event)
algoProb.setMaximumOuterSampling(1000)
algoProb.setMaximumCoefficientOfVariation(0.01)
algoProb.run()

In [ ]:
# Get the results
resultAlgo = algoProb.getResult()
neval = g.getEvaluationCallsNumber()
print("Number of function calls = %d" % (neval))
pf = resultAlgo.getProbabilityEstimate()
print("Failure Probability = %.4f" % (pf))
level = 0.95
c95 = resultAlgo.getConfidenceLength(level)
pmin = pf - 0.5 * c95
pmax = pf + 0.5 * c95
print("%.1f %% confidence interval :[%.4f,%.4f] " % (level * 100, pmin, pmax))

## Compute the bounds of the domain

In [ ]:
inputVector = event.getAntecedent()
distribution = inputVector.getDistribution()
X1 = distribution.getMarginal(0)
X2 = distribution.getMarginal(1)
alphaMin = 0.00001
alphaMax = 1 - alphaMin
lowerBound = ot.Point(
    [X1.computeQuantile(alphaMin)[0], X2.computeQuantile(alphaMin)[0]]
)
upperBound = ot.Point(
    [X1.computeQuantile(alphaMax)[0], X2.computeQuantile(alphaMax)[0]]
)

In [ ]:
nbPoints = [100, 100]
figure = g.draw(lowerBound, upperBound, nbPoints)
figure.setTitle(" Iso-values of limit state function")
figure

## Print the iso-values of the distribution

In [ ]:
distribution.drawPDF()

In [ ]:
sampleSize = 5000
sampleInput = inputVector.getSample(sampleSize)
sampleOutput = g(sampleInput)
drawEvent = otb.DrawEvent(event)

In [ ]:
cloud = drawEvent.drawSample(sampleInput, sampleOutput)
cloud

## Draw the limit state surface

In [ ]:
bounds = ot.Interval(lowerBound, upperBound)
bounds

In [ ]:
nX = 100
nY = 100
graph = drawEvent.drawLimitState(bounds, nX, nY)
graph.add(cloud)
graph

In [ ]:
domain = drawEvent.fillEvent(bounds)
domain

In [ ]:
domain.add(cloud)
domain